# Data Cleaning: MEPS Medical Conditions Data (2014-2019)

## Import packages and read data

In [1]:
# Import statements
import pandas as pd
import numpy as np

# To show all lines of output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [80]:
# Reading in the data
data_to_read = ['MC_14.csv', 'MC_15.csv', 'MC_16.csv', 'MC_17.csv', 'MC_18.csv', 'MC_19.csv']
years_to_read = ['2014', '2015', '2016', '2017', '2018', '2019']
mc_cols = ['DUPERSID', 'CONDN', 'INJURY']

## View one data year

In [105]:
data_raw = pd.read_csv('MC/MC_19.csv')

In [106]:
# Describing the data
data_raw.describe()
data_raw.columns

# Filter out only columns we need
df = data_raw[mc_cols]

,DUID,PID,DUPERSID,CONDN,CONDIDX,HHNUM,IPNUM,OPNUM,OBNUM,ERNUM,RXNUM,PERWT_YEARF,VARSTR,VARPSU,year
count,8.756100e+04,87561.000000,8.756100e+04,87561.000000,8.756100e+04,87561.000000,87561.000000,87561.000000,87561.000000,87561.000000,87561.000000,87561.000000,87561.000000,87561.000000,87561.0
mean,2.391797e+06,102.762588,2.391797e+09,6.187252,2.391797e+12,0.173742,0.040223,0.264970,2.302395,0.079864,1.626375,10934.329965,2055.463563,1.851772,19.0
std,6.995006e+04,12.319195,6.995006e+07,5.334625,6.995006e+10,1.237583,0.232921,2.093283,7.488539,0.348568,2.198368,6711.337645,33.712282,0.904541,0.0
min,2.320002e+06,101.000000,2.320002e+09,1.000000,2.320002e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2001.000000,1.000000,19.0
25%,2.324788e+06,101.000000,2.324788e+09,2.000000,2.324788e+12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6640.465438,2026.000000,1.000000,19.0
50%,2.329271e+06,101.000000,2.329271e+09,5.000000,2.329271e+12,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,9564.524517,2052.000000,2.000000,19.0
75%,2.464603e+06,102.000000,2.464603e+09,8.000000,2.464603e+12,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,13629.531798,2084.000000,2.000000,19.0
max,2.469689e+06,503.000000,2.469689e+09,54.000000,2.469689e+12,60.000000,6.000000,141.000000,499.000000,15.000000,95.000000,77911.063903,2117.000000,6.000000,19.0


Index(['DUID', 'PID', 'DUPERSID', 'CONDN', 'CONDIDX', 'PANEL', 'CONDRN',
       'AGEDIAG', 'CRND1', 'CRND2', 'CRND3', 'CRND4', 'CRND5', 'INJURY',
       'ACCDNWRK', 'ICD10CDX', 'CCSR1X', 'CCSR2X', 'CCSR3X', 'HHNUM', 'IPNUM',
       'OPNUM', 'OBNUM', 'ERNUM', 'RXNUM', 'PERWT_YEARF', 'VARSTR', 'VARPSU',
       'year'],
      dtype='object')

In [107]:
# List of column names
list(df.columns)

# Number of unique IDs
print(f"Num of unique DUPERSID: {df['DUPERSID'].nunique()}")

# Length of original dataset
print(f"Length of dataset: {len(df)}")

# Check value counts of injury
print("Value counts for Injury:")
df['INJURY'].value_counts(dropna=False)

print(f"Class type for Injury var: {type(df['INJURY'][0])}")


['DUPERSID', 'CONDN', 'INJURY']

Num of unique DUPERSID: 19810
Length of dataset: 87561
Value counts for Injury:


2 NO     83051
1 YES     4510
Name: INJURY, dtype: int64

Class type for Injury var: <class 'str'>


## Data cleaning (for one year)

#### Recoding Injury from 1/2 to 1/0

In [108]:
# Change INJURY from 1/2 to 1/0
# df['INJURY'] = df['INJURY'].apply(lambda x: 1 if x == 1 else 0)
df['INJURY'] = df['INJURY'].apply(lambda x: 1 if x == "1 YES" else 0)

# Check recoded value counts of injury - good
df['INJURY'].value_counts(dropna=False)

<ipython-input-108-7f678b7c23da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INJURY'] = df['INJURY'].apply(lambda x: 1 if x == "1 YES" else 0)


0    83051
1     4510
Name: INJURY, dtype: int64

#### Aggregating data into a new dataframe

In [99]:
# Aggregate data by DUPERSID
df_agg = df.groupby(by="DUPERSID", as_index=False).agg(
    # Create new vars
    NUM_CONDITIONS = ('CONDN', 'count'),
    INJURY = ('INJURY', 'max'))

#### Checking data cleaning

In [100]:
# Check new column names
list(df_agg.columns)

# Check length of agg df = # of unique ID's
len(df_agg)
df_agg['DUPERSID'].nunique()

# Check that # unique IDs stayed the same in agg df
df['DUPERSID'].nunique() == df_agg['DUPERSID'].nunique()

# Check that sum of # conditions = number of rows of original df
sum(df_agg["NUM_CONDITIONS"]) == len(df)

# See frequencies of "INJURY"
df_agg['INJURY'].value_counts()

# Checking against original injury var
ct_inj_1 = df.loc[df['INJURY'] == 1]['DUPERSID'].nunique()
ct_inj_1
df_agg['INJURY'].value_counts()[1] == ct_inj_1

['DUPERSID', 'NUM_CONDITIONS', 'INJURY']

23936

23936

True

True

0    18906
1     5030
Name: INJURY, dtype: int64

5030

True